In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Nlp_Customer_ticket

/content/drive/MyDrive/Nlp_Customer_ticket


In [55]:
pip install dvc[gdrive]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.3/77.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.8/438.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8

In [46]:
from google.colab import userdata
GITHUB_TOKEN = userdata.get('GIT_NLP_CUSTOM')

In [47]:
!git remote set-url origin https://github.com/Jojo225consulting/Nlp_Customer_ticket.git

In [ ]:
#!git remote add origin https://$GITHUB_TOKEN@github.com/Jojo225consulting/Nlp_Customer_ticket.git

In [ ]:
!git remote -v

origin	https://github.com/Jojo225consulting/Nlp_Customer_ticket.git (fetch)
origin	https://github.com/Jojo225consulting/Nlp_Customer_ticket.git (push)


In [ ]:
!git pull https://$GITHUB_TOKEN@github.com/Jojo225consulting/Nlp_Customer_ticket.git

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 1.90 MiB | 2.85 MiB/s, done.
From https://github.com/Jojo225consulting/Nlp_Customer_ticket
 * branch            HEAD       -> FETCH_HEAD
Updating bf9e8ef..e57da01
error: Your local changes to the following files would be overwritten by merge:
	Exploration.ipynb
Please commit your changes or stash them before you merge.
Aborting


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Exploration.ipynb
	modified:   Mlflow_ui.ipynb
	modified:   Nlp_customer_ticket.ipynb
	modified:   Operations.ipynb
	modified:   packages_created/__pycache__/preprocessing_data.cpython-311.pyc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	config.cfg
	mlruns/
	output/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git branch

* main
  master


In [ ]:
!git branch -r

  origin/main


In [ ]:
!git log

commit bf9e8efbf5f008b6c7ebf14bc6f881d6f31b5181 (HEAD -> main, origin/main)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Wed Jul 23 01:42:40 2025 +0000

    Adding file Mlflow_UI with ngrok and adding file Operations, a old part of NLP_Customer_ticket

commit a62c173b963c7fbf73a0aeae71e72d4a3d099bef (master)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Mon Jul 21 20:38:00 2025 +0000

    First versions of data

commit aa62633ae061f46157d77c43574aea3d284a3787
Author: jojo225 <josuenassa@yahoo.fr>
Date:   Mon Jul 21 20:52:12 2025 +0200

    reinit in dropping manually on repo

commit f04f8dc7d128d4d4cf39f784749bf1579f4beffe
Author: jojo225 <josuenassa@yahoo.fr>
Date:   Mon Jul 21 16:45:53 2025 +0200

    data, notebook and modules

commit 3cc9690a776c4e8f4e5752306d4276da25200a53
Author: Jojo225consulting <consulting.josue@gmail.com>
Date:   Mon Jul 21 16:35:06 2025 +0200

    Update README.md

commit 5aa87f85db0e25d3060994d32b898733b40ba912
Author: Jojo2

In [ ]:
!git checkout -b LDA_model

Switched to a new branch 'LDA_model'


In [ ]:
!git branch

* LDA_model
  main
  master


In [ ]:
!git log

commit bf9e8efbf5f008b6c7ebf14bc6f881d6f31b5181 (HEAD -> LDA_model, origin/main, main)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Wed Jul 23 01:42:40 2025 +0000

    Adding file Mlflow_UI with ngrok and adding file Operations, a old part of NLP_Customer_ticket

commit a62c173b963c7fbf73a0aeae71e72d4a3d099bef (master)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Mon Jul 21 20:38:00 2025 +0000

    First versions of data

commit aa62633ae061f46157d77c43574aea3d284a3787
Author: jojo225 <josuenassa@yahoo.fr>
Date:   Mon Jul 21 20:52:12 2025 +0200

    reinit in dropping manually on repo

commit f04f8dc7d128d4d4cf39f784749bf1579f4beffe
Author: jojo225 <josuenassa@yahoo.fr>
Date:   Mon Jul 21 16:45:53 2025 +0200

    data, notebook and modules

commit 3cc9690a776c4e8f4e5752306d4276da25200a53
Author: Jojo225consulting <consulting.josue@gmail.com>
Date:   Mon Jul 21 16:35:06 2025 +0200

    Update README.md

commit 5aa87f85db0e25d3060994d32b898733b40ba912
Au

In [ ]:
#!git pull --rebase origin main
#!git pull

In [3]:
import polars as pl
import pandas as pd
import numpy as np

In [4]:

import sys
sys.path.append("/content/drive/MyDrive/Nlp_Customer_ticket/packages_created")

In [5]:
import preprocessing_data as ppd

In [6]:
import spacy
from spacy.tokens import DocBin

In [30]:
df=ppd.make_clean("/content/drive/MyDrive/Nlp_Customer_ticket/Data/Raw/aa_dataset-tickets-multi-lang-5-2-50-version_new_features_train.csv")
df_train = df.filter(pl.col("language") == "en")

df_valid = ppd.make_clean("/content/drive/MyDrive/Nlp_Customer_ticket/Data/Raw/aa_dataset-tickets-multi-lang-5-2-50-version_new_features_valid.csv")
df_valid=df_valid.filter(pl.col("language")=="en")

No difference between after cleaning
No difference between after cleaning


In [31]:
#Unique_queue = {'Technical Support', 'Product Support','Customer Service', 'IT Support'}
df_train=pd.DataFrame(df_train, columns=df_train.columns)
df_valid=pd.DataFrame(df_valid, columns=df_valid.columns)
#df_train["queue_treated"]=df_train['queue'].apply(lambda x: x if x in Unique_queue else 'Others')
#df_test["queue_treated"]=df_test['queue'].apply(lambda x: x if x in Unique_queue else 'Others')

In [ ]:
#df_train["queue_treated"].value_counts()*100/sum(df_train["queue_treated"].value_counts())

,count
queue_treated,
Technical Support,28.993757
Others,25.560044
Product Support,18.808912
Customer Service,14.750888
IT Support,11.886400


In [ ]:
#df_test["queue_treated"].value_counts()*100/sum(df_test["queue_treated"].value_counts())

,count
queue_treated,
Technical Support,33.669065
Others,22.158273
Product Support,16.834532
Customer Service,15.323741
IT Support,12.014388


In [33]:
df_train["topic_body_detail"].unique()

array(['Bugs logiciels et redémarrage',
       'Demandes d’information et service client',
       'Problèmes de facturation et comptes',
       'Intégration de solutions SaaS',
       'Stratégie marketing et visibilité',
       'Analyse financière et décisions stratégiques',
       'Problèmes de réseau et dispositifs',
       'Sécurité des données médicales'], dtype=object)

In [34]:
labels = list(df_train["topic_body_detail"].unique())

TRAIN_DATA = []
for _, row in df_train.iterrows():
    cats = {label: 1.0 if label == row["topic_body_detail"] else 0.0 for label in labels}
    TRAIN_DATA.append((row["body"], {"cats": cats}))


In [35]:
labels = list(df_valid["topic_body_detail"].unique())

VALID_DATA = []
for _, row in df_valid.iterrows():
    cats = {label: 1.0 if label == row["topic_body_detail"] else 0.0 for label in labels}
    VALID_DATA.append((row["body"], {"cats": cats}))


In [36]:
def convert_to_spacy(data, output_file):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, annotations in data:
        doc = nlp.make_doc(text)
        doc.cats = annotations["cats"]
        db.add(doc)
    db.to_disk(output_file)

convert_to_spacy(TRAIN_DATA, "train_new_features_topics.spacy")
convert_to_spacy(VALID_DATA, "valid_new_features_topics.spacy")


In [37]:
!cp -r /content/drive/MyDrive/Nlp_Customer_ticket/train_new_features_topics.spacy /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format
!cp -r /content/drive/MyDrive/Nlp_Customer_ticket/valid_new_features_topics.spacy /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format

In [38]:
!ls /content/drive/MyDrive/Nlp_Customer_ticket/Data/Spacy_format

test_queue_gathering.spacy	 train.spacy
test.spacy			 valid_new_features_topics.spacy
train_new_features_topics.spacy  valid_queue_gathering.spacy
train_queue_gathering.spacy	 valid.spacy


In [51]:
!git restore Exploration.ipynb


In [52]:
!git pull https://$GITHUB_TOKEN@github.com/Jojo225consulting/Nlp_Customer_ticket.git LDA_model

From https://github.com/Jojo225consulting/Nlp_Customer_ticket
 * branch            LDA_model  -> FETCH_HEAD
Updating 764d0c0..e6fff3e
Fast-forward
 Exploration.ipynb | 39985 +++++++++++++++++++++++++++++++++++++---------------
 1 file changed, 28445 insertions(+), 11540 deletions(-)


In [53]:
!git status

On branch LDA_model
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Mlflow_ui.ipynb
	modified:   Nlp_customer_ticket.ipynb
	modified:   Operations.ipynb
	modified:   packages_created/__pycache__/preprocessing_data.cpython-311.pyc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	config.cfg
	mlruns/
	output/
	train_new_features_topics.spacy
	valid_new_features_topics.spacy

no changes added to commit (use "git add" and/or "git commit -a")


In [56]:
!dvc add Data

⠋ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
          |0.00 [00:00,     ?file/s]
 14% 1.00/7.00 [00:00<00:01, 4.12file/s{'info': ''}]
 29% 2.00/7.00 [00:01<00:04, 1.04file/s{'info': ''}]
 43% 3.00/7.00 [00:02<00:03, 1.30file/s{'info': ''}]
 57% 4.00/7.00 [00:02<00:02, 1.43file/s{'info': ''}]
 71% 5.00/7.00 [00:03<00:01, 1.53file/s{'info': ''}]
 53% 8.00/15.0 [00:03<00:02, 2.73file/s{'info': ''}]
 60% 9.00/15.0 [00:04<00:02, 2.84file/s{'info': ''}]
 67% 10.0/15.0 [00:04<00:01, 2.86file/s{'info': ''}]
 73% 11.0/15.0 [00:04<00:01, 3.02file/s{'info': ''}]
 80% 12.0/15.0 [00:05<00:01, 2.96file/s{'info': ''}]
 87% 13.0/15.0 [00:05<00:00, 3.25file/s{'info': ''}]
                                                    
!
  0% |          |0/? [00:00<?,    ?files/s]
 12% 2/16 [00:00<00:02,  5.15files/s{'info': ''}]
 19% 3/16 [00:01<00:04,  2.64files/s{'info': ''}]
 25% 4/16 [00:01<00:04,  2.91files/s{'info': ''}]
 38% 6/16 [00:02<00:05,  1.98files/s{'info': ''}]
 44% 7/16 [

In [57]:
!git status

On branch LDA_model
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Data.dvc
	modified:   Mlflow_ui.ipynb
	modified:   Nlp_customer_ticket.ipynb
	modified:   Operations.ipynb
	modified:   packages_created/__pycache__/preprocessing_data.cpython-311.pyc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	config.cfg
	mlruns/
	output/
	train_new_features_topics.spacy
	valid_new_features_topics.spacy

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add Data.dvc

In [ ]:
!git status

On branch LDA_model
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Data.dvc

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Exploration.ipynb
	modified:   Mlflow_ui.ipynb
	modified:   Nlp_customer_ticket.ipynb
	modified:   Operations.ipynb
	modified:   packages_created/__pycache__/preprocessing_data.cpython-311.pyc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	config.cfg
	mlruns/
	output/



In [ ]:
!git commit -m "Adding new data train et valid containing new features topic_body"

[LDA_model 79e92c4] Adding new data train et valid containing new features topic_body
 1 file changed, 3 insertions(+), 3 deletions(-)


In [ ]:
!git status

On branch LDA_model
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   Operations.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Exploration.ipynb
	modified:   Mlflow_ui.ipynb
	modified:   Nlp_customer_ticket.ipynb
	modified:   packages_created/__pycache__/preprocessing_data.cpython-311.pyc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.gitignore
	config.cfg
	mlruns/
	output/



In [ ]:
!git push https://$GITHUB_TOKEN@github.com/Jojo225consulting/Nlp_Customer_ticket.git LDA_model

Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 4.85 KiB | 207.00 KiB/s, done.
Total 6 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
remote: This repository moved. Please use the new location:
remote:   https://github.com/Jojo225consulting/Nlp_customer_ticket.git
remote: 
remote: Create a pull request for 'LDA_model' on GitHub by visiting:
remote:      https://github.com/Jojo225consulting/Nlp_customer_ticket/pull/new/LDA_model
remote: 
To https://github.com/Jojo225consulting/Nlp_Customer_ticket.git
 * [new branch]      LDA_model -> LDA_model


In [ ]:
!git log

commit 764d0c09c0044c32e1cc389fd50975d1115eae63 (HEAD -> LDA_model)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Thu Jul 31 23:23:46 2025 +0000

    file Operations

commit 79e92c4b97079fa74dc2665ad29f94a667c6b205
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Thu Jul 31 23:20:49 2025 +0000

    Adding new data train et valid containing new features topic_body

commit bf9e8efbf5f008b6c7ebf14bc6f881d6f31b5181 (origin/main, main)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Wed Jul 23 01:42:40 2025 +0000

    Adding file Mlflow_UI with ngrok and adding file Operations, a old part of NLP_Customer_ticket

commit a62c173b963c7fbf73a0aeae71e72d4a3d099bef (master)
Author: Achi Josué NASSA <consulting.josue@gmail.com>
Date:   Mon Jul 21 20:38:00 2025 +0000

    First versions of data

commit aa62633ae061f46157d77c43574aea3d284a3787
Author: jojo225 <josuenassa@yahoo.fr>
Date:   Mon Jul 21 20:52:12 2025 +0200

    reinit in dropping manually on re